# 5장: 출력 형식 지정 및 Claude를 위한 말하기

- [수업](#lesson)
- [연습 문제](#exercises)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# 미리 채울 텍스트를 위한 새 인수가 추가되었으며, 기본값은 빈 문자열입니다
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 수업

**Claude는 다양한 방식으로 출력 형식을 지정할 수 있습니다**. 그렇게 하도록 요청하기만 하면 됩니다!

이러한 방법 중 하나는 XML 태그를 사용하여 응답을 다른 불필요한 텍스트와 분리하는 것입니다. 이미 XML 태그를 사용하여 프롬프트를 Claude에게 더 명확하고 구문 분석하기 쉽게 만들 수 있다는 것을 배웠습니다. 또한 Claude에게 **XML 태그를 사용하여 출력을 더 명확하고 인간이 더 쉽게 이해할 수 있도록** 만들도록 요청할 수도 있습니다.

### 예제

2장에서 서문을 완전히 건너뛰도록 Claude에게 요청하여 해결한 '시 서문 문제'를 기억하시나요? 또한 **Claude에게 시를 XML 태그 안에 넣으라고 말하여** 비슷한 결과를 얻을 수도 있습니다.

In [ ]:
# 변수 내용
ANIMAL = "토끼"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 작성해주세요. <haiku> 태그 안에 넣어주세요."

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

왜 이것이 우리가 원하는 것일까요? 출력을 XML 태그로 감싸면 **최종 사용자가 XML 태그 사이의 내용을 추출하는 짧은 프로그램을 작성하여 시만 안정적으로 얻을 수 있기 때문입니다**.

이 기술의 확장은 **첫 번째 XML 태그를 `assistant` 차례에 넣는 것입니다. `assistant` 차례에 텍스트를 넣으면, 기본적으로 Claude에게 Claude가 이미 무언가를 말했고, 그 지점부터 계속해야 한다고 말하는 것입니다. 이 기술을 "Claude를 위한 말하기" 또는 "Claude의 응답 미리 채우기"라고 합니다.**

아래에서는 첫 번째 `<haiku>` XML 태그로 이 작업을 수행했습니다. Claude가 우리가 멈춘 지점에서 직접 계속하는 것을 확인하세요.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 작성해주세요. <haiku> 태그 안에 넣어주세요."

# Claude의 응답 미리 채우기
PREFILL = "<haiku>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude는 또한 `JSON`과 같은 다른 출력 형식 스타일에서도 뛰어납니다. JSON 출력을 강제하려면(결정적이지는 않지만 거의 비슷하게), 여는 중괄호 `{`로 Claude의 응답을 미리 채울 수도 있습니다.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 작성해주세요. 키가 "first_line", "second_line", "third_line"인 JSON 형식을 사용하세요."

# Claude의 응답 미리 채우기
PREFILL = "{"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

아래는 **동일한 프롬프트에 여러 입력 변수와 출력 형식 지정을 모두 XML 태그를 사용하여 수행하는** 예입니다.

In [ ]:
# 첫 번째 입력 변수
EMAIL = "안녕 잭, 네가 작성하기로 한 프롬프트에 대한 빠른 업데이트를 요청하려고 연락했어."

# 두 번째 입력 변수
ADJECTIVE = "중세 영어"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"안녕 Claude. 여기 이메일이 있어: <email>{EMAIL}</email>. 이 이메일을 더 {ADJECTIVE} 스타일로 만들어줘. 새 버전을 <{ADJECTIVE}_email> XML 태그 안에 작성해줘."

# Claude의 응답 미리 채우기(이제 변수가 있는 f-문자열로)
PREFILL = f"<{ADJECTIVE}_email>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 보너스 수업

API를 통해 Claude를 호출하는 경우, 닫는 XML 태그를 `stop_sequences` 매개변수에 전달하여 원하는 태그를 내보낸 후 Claude가 샘플링을 중지하도록 할 수 있습니다. 이렇게 하면 이미 원하는 답변을 제공한 후 Claude의 결론적인 발언을 제거하여 비용과 마지막 토큰까지의 시간을 절약할 수 있습니다.

---

## 연습 문제

- [연습 문제 5.1 - 스테프 커리 GOAT](#exercise-51---steph-curry-goat)
- [연습 문제 5.2 - 두 개의 하이쿠](#exercise-52---two-haikus)
- [연습 문제 5.3 - 두 개의 하이쿠, 두 마리의 동물](#exercise-53---two-haikus-two-animals)

### 연습 문제 5.1 - 스테프 커리 GOAT

선택을 강요받으면, Claude는 마이클 조던을 역대 최고의 농구 선수로 지정합니다. Claude가 다른 사람을 선택하도록 할 수 있을까요?

`PREFILL` 변수를 변경하여 **Claude가 스테픈 커리가 역대 최고의 농구 선수라는 상세한 주장을 하도록 강제하세요**. 이 연습의 초점이므로 `PREFILL` 외에는 아무것도 변경하지 마세요.

In [ ]:
# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"역대 최고의 농구 선수는 누구인가요? 한 명의 특정 선수를 선택해주세요."

# Claude의 응답 미리 채우기
PREFILL = ""

# Claude의 응답 가져오기
response = get_completion(PROMPT, prefill=PREFILL)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 연습 문제 5.2 - 두 개의 하이쿠

XML 태그를 사용하여 아래의 `PROMPT`를 수정하여 Claude가 한 개가 아닌 두 개의 하이쿠를 동물에 대해 작성하도록 하세요. 한 시가 끝나고 다른 시가 시작되는 지점이 명확해야 합니다.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 작성해주세요. <haiku> 태그 안에 넣어주세요."

# Claude의 응답 미리 채우기
PREFILL = "<haiku>"

# Claude의 응답 가져오기
response = get_completion(PROMPT, prefill=PREFILL)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 연습 문제 5.3 - 두 개의 하이쿠, 두 마리의 동물

아래의 `PROMPT`를 수정하여 **Claude가 두 마리의 다른 동물에 대한 두 개의 하이쿠를 생성하도록** 하세요. 첫 번째 대체를 위해 `{ANIMAL1}`을 자리 표시자로 사용하고, 두 번째 대체를 위해 `{ANIMAL2}`를 자리 표시자로 사용하세요.

In [ ]:
# 첫 번째 입력 변수
ANIMAL1 = "고양이"

# 두 번째 입력 변수
ANIMAL2 = "개"

# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL1}에 대한 하이쿠를 작성해주세요. <haiku> 태그 안에 넣어주세요."

# Claude의 응답 가져오기
response = get_completion(PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### 축하합니다!

지금까지 모든 연습 문제를 해결했다면, 다음 장으로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!